# Initialize

In [ ]:
# pip install crewai
import os
import sys
import subprocess

from crewai import Agent, Task, Crew
from crewai.tools import tool
from dotenv import load_dotenv

# The agent asks the developer a program file. 
# {Program}{Language}
# If it's a C++ code, the agent will add the FidesZKP.h library to the user's program.cpp program and will save it as program_AddedFidesZKPLib.cpp 
# If it's a Python code, the agent will add the Fides Python package to the user's program.py program and will save it as program_AddedFidesZKPLib.py 
# 
# {Processor} 
# Then, the agent asks the developer to choose a target processor including RISC-V and ARM. 
# Then, the agent compiles the code using g++ compiler for the target processor to generate an assembly file; program_AddedFidesZKP.s
# Then, the agent executes the commitmentGenerator file 
# to create program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, program_AddedFidesZKP_Commitment.json
# 
# The agent returns program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json files to the developer.
# The agent uploads the program_AddedFidesZKP_Commitment.json on the Fides Innova blockchain using https://rpc.fidesinnova.io API and its own wallet.
# 
# The agent return a link to the explorer to show the submitted commitment file, https://explorer.fidesinnova.io/xxxxxxxxxx
# The Fides Innova explorer shows the "CommitmentManagerAIAgent" name as the transaction submitter on the blockchain. 

# --- Step 2: Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Device Config Agent

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)

# deviceConfigTool = tool()
# deviceConfigAgent = Agent(role="commitmentGenerator")
# deviceConfigTask = Task()

@tool("deviceConfigTool")
def deviceConfigTool(myJson: str) -> str:
    """
    Save the input myJson (stringified JSON) into device_config.json.
    """
    import json

    try:
        parsed = json.loads(myJson)
    except json.JSONDecodeError:
        return "Error: Provided input is not valid JSON."

    with open("device_config.json", "w+") as f:
        json.dump(parsed, f, indent=2)

    return "✅ Device config file updated successfully."

deviceConfigAgent = Agent(
    role="device_config",
    goal=(
        "Create a JSON object called myJson with the following keys:\n"
        "- class: {class}\n"
        "- deviceType: {deviceType}\n"
        "- deviceIdType: {deviceIdType}\n"
        "- deviceModel: {deviceModel}\n"
        "- manufacturer: {manufacturer}\n"
        "- softwareVersion: {softwareVersion}\n"
        "- code_block: {code_block}\n"
        "Then call deviceConfigTool(myJson) to save it."
    ),
    backstory="You are responsible for creating device configs based on user inputs.",
    tools=[deviceConfigTool]
)

deviceConfigTask = Task(
    agent=deviceConfigAgent,
    description=(
        "Generate a JSON object called myJson with the specified keys and values. "
        "Then invoke deviceConfigTool(myJson) to save it to device_config.json."
    ),
    expected_output="Print a message whether the device config file is updated or not."
)

# Compiler Agent

In [ ]:
@tool("CompilerTool")
def compilerTool(program: str) -> str:
    """
    Read the user program file, recognize the program language.
    Add new lines to the program to import the FidesZKP library or package.
    Add '_AddedFidesZKP' to the program name and save it.
    Compile the program and return the assembly filename if successful.
    If compilation fails, raise an Exception to stop the crew.
    """
    filename = os.path.basename(program)
    base_name, ext = os.path.splitext(filename)
    ext = ext.lower()
    filename = base_name + ext

    if ext == ".py":
        language = "Python"
    elif ext == ".cpp":
        language = "C++"
    else:
        raise ValueError("Unsupported file extension")

    with open(program, "r") as f1:
        program_code = f1.read()

    if language == "C++":
        new_line = '#include "lib/fidesinnova.h"\n'
        if not (new_line in program_code):
            modified_code = new_line + program_code
        else:
            modified_code = program_code
        new_filename = f"{base_name}_AddedFidesZKPLib.cpp"

        with open(new_filename, "w+") as f2:
            f2.write(modified_code)

        asm_filename = f"{base_name}_AddedFidesZKPLib.s"
        command_to_execute = f"g++ -std=c++17 -S {new_filename} -o {asm_filename} -lstdc++"
        result = subprocess.run(command_to_execute.split(), capture_output=True, text=True)

        if result.returncode != 0:
            # Raise an exception to stop the crew and print error
            raise Exception(
                f"Compilation failed!\nError Output:\n{result.stderr}\n"
                "Please check your code for syntax errors or missing dependencies. "
                "Fix the issues and try again."
            )

        return asm_filename

    else:
        raise NotImplementedError("Only C++ is supported for now")

CompilerAgent = Agent(
    role="Compiler",
    goal="Execute the compilerTool tool. The default {program} filename is program.cpp if it's not set by the user. If there is any error, print the error for the user and provide some suggestions to fix it. Then, do not proceed with any other agent and stop the crew.",
    backstory="",
    tools=[compilerTool],
    allow_delegation=True,
)

CompileTask = Task(
    agent=CompilerAgent,
    description="Process the user's program file {program} with compilerTool.",
    expected_output=(
        "If there is no error from the g++ compilation command, then return only the assembly output filename resulting from execution of the g++ compiler. "
        "If there is any error, print the error for the user and provide some suggestions to fix it. Then, do not proceed with any other agent and stop the crew."
    ),
)

# Commitment Generator Agent

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# commitmentGeneratorTool = tool()
# commitmentGeneratorAgent = Agent(role="commitmentGenerator")
# commitmentGeneratorTask = Task()

# execute:
# commitmentGenerator-ubuntu program_AddedFidesZKPLib.s
# commitmentGenerator-Mac program_AddedFidesZKPLib.s

# Role, Goal, Backstory, LLm instance, Tool (optional)
@tool("operatingSystemTool")
def operatingSystemTool() -> str:
    """
    Find out what the operating system is. This must be executed to let the agent learn about the operating system name.
    """
    if sys.platform.startswith('linux'):
        # Further check for Ubuntu specifically
        if os.path.exists('/etc/lsb-release'):
            with open('/etc/lsb-release', 'r') as f:
                content = f.read()
                if 'DISTRIB_ID=Ubuntu' in content:
                    return "Ubuntu"
                else:
                    return "Linux (Other)"
        else:
             return "Linux (Likely)"
    elif sys.platform == 'darwin':
        return "macOS"
    else:
        return "Other OS"

@tool("commitmentGeneratorOnMacTool")
def commitmentGeneratorOnMacTool(program: str) -> str:
    """
    Execute ./commitmentGenerator using the output of the Compiler agent output. This execution is for macOS machines.
    """
    os.system(f"./commitmentGenerator-Mac {program}")
    return True

@tool("commitmentGeneratorOnUbuntuTool")
def commitmentGeneratorOnUbuntuTool(program: str) -> str:
    """
    Execute ./commitmentGenerator using the output of the Compiler agent output. This execution is for Ubuntu machines.
    """
    os.system(f"./commitmentGenerator-ubuntu {program}")
    return True


commitmentGeneratorAgent = Agent(role="commitmentGenerator-executor", 
                                 goal = "Based on the return value of operatingSystemTool, find out which computer's operation system is currently running on the computer. If it's a macOS, execute commitmentGeneratorOnMacTool tool. The input for this tool is the output filename from the Compiler agent. If it's a Ubuntu, execute commitmentGeneratorOnUbuntuTool tool. The input for this tool is the output filename from the Compiler agent. Otherwise, tell the user that yuu support only macOS and Ubuntu platform at the moment and you do not have a tool for the current operating system.",
                                 backstory="", 
                                 tools = [operatingSystemTool, commitmentGeneratorOnMacTool, commitmentGeneratorOnUbuntuTool]
                                 )
FindingOperatingSystemTask = Task( agent = commitmentGeneratorAgent, 
                               description = "Call the operatingSystemTool with the output of the Compiler agent as the input. Then, based on the output operatingSystemTool, determine the type of operating system.", 
                               expected_output="Operating System type" 
                               )
ExecutingCommitmentGeneratorTask = Task( agent = commitmentGeneratorAgent, 
                               description = "Based on the output of the operatingSystemTool tool, choose which tool, commitmentGeneratorOnUbuntuTool or commitmentGeneratorOnLinuxTool or none, must be executed.", 
                               expected_output="True or False" 
                               )

# Verifiable Computing Crew

In [ ]:
# Agent, Description, Expected Output   
agents = [deviceConfigAgent, CompilerAgent,  commitmentGeneratorAgent]
tasks =  [deviceConfigTask,  CompileTask,    FindingOperatingSystemTask, ExecutingCommitmentGeneratorTask]
crew1 = Crew( agents = agents, tasks= tasks, verbose = True)

# Test

In [ ]:
def query():
    question = input("What's your program name? (default: program.cpp)")
    program_class = input("What's your program ZKP class? (default: 1)")
    deviceType = input("What's your program device type? (default: Sensor)")
    deviceIdType = input("What's your device ID type? (default: MAC)")
    deviceModel = input("What's your device model? (default: Siemens IOT2050)")
    manufacturer = input("What's your brand name? (default: Fides)")
    softwareVersion = input("What's your program/firmware version? (default: 1.0)")
    code_block = input("What's your program/firmware version? (default: [14, 15]])")

    result = crew1.kickoff({"program":question, 
                            "class": program_class or "1",
                            "deviceType": deviceType or "sensor",
                            "deviceIdType": deviceIdType or "MAC",
                            "deviceModel": deviceModel or "Siemens IOT2050",
                            "manufacturer": manufacturer or "FDS",
                            "softwareVersion": softwareVersion or "1.0",
                            "code_block": code_block or "[14, 15]"
                            })
    return result

query()

In [ ]:
# @tool("commitmentSubmitterTool")

# def commitmentSubmitterTool(program):
#     """Read the user program__AddedFidesZKP.s file, and execute the ./commitmentGenerator file with the program__AddedFidesZKP.s file to generate three files including program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, and program_AddedFidesZKP_Commitment.json"""
#         # Outputs:
#         # 1- program_AddedFidesZKP_ZKPGen.s 
#         # 2- program_AddedFidesZKP_Param.json 
#         # 3- program_AddedFidesZKP_Commitment.json
#     if ".s" in program:
#         Language = "assembly"
#     else:
#         Language = "Unknown"

#     if Language == "assembly":
#         os.system(f"./commitmentGenerator {program}")
#         return "The new files are: " + program.split('.')[0]+"_ZKPGen.s," + program.split('.')[0]+"_Param.json, and "  + program.split('.')[0]+"_Commitment.json."
#     else:
#         raise NotImplementedError    
 
#  def blockchainUploader(program):
#     """Upload program_AddedFidesZKP_Commitment.json on fides Innova blockchain"""
#         # program_AddedFidesZKP_Commitment.json
#     if ".s" in program:
#         Language = "assembly"
#     else:
#         Language = "Unknown"

#     if Language == "assembly":
#         os.system(f"./commitmentGenerator {program}")
#         return "The transaction id: " + program.split('.')[0]+"_ZKPGen.s,"
#     else:
#         raise NotImplementedError   
    
# commitmentSubmitterAgent = Agent(role="commitmentGenerator", goal="Read the user program file {program}, and execute commitmentGenerator file. It generates three output files. The firs file is program_AddedFidesZKP_ZKPGen.s that has some extra opcodes compared to the program_AddedFidesZKP.s file. These lines call the proof_generation function from the FidesZKP library or package. The second file is program_AddedFidesZKP_Param.json that contains some pre-calculated numbers to accelerate the proof generation function in production. The third one is a commitment file that shows the arithmetic version of the program, program_AddedFidesZKP_Commitment.json, and should be uploaded on the Fides blockchain. This file will be used later in verification purposes by the proof verifier crew.",
#                       backstory="", tools = [commitmentGeneratorTool])
# commitmentSubmitterTask = Task( agent = commitmentGeneratorAgent, description = "execute the user's program file {program} with commitmentGenerator, and upload the generated commitment file on Fides Innova blockchain", expected_output="Two files, program_AddedFidesZKP_ZKPGen.s and program_AddedFidesZKP_Param.json to be delivered to the developer. One file, program_AddedFidesZKP_Commitment.json , to be uploaded on the Fides blockchain." )

